# Image Processing

In [1]:
import numpy as np#used for numerical analysis
import tensorflow #open source used for both ML and DL for computation
from tensorflow.keras.models import Sequential #it is a plain stack of layers
from tensorflow.keras import layers #A layer consists of a tensor-in tensor-out computation function
#Dense layer is the regular deeply connected neural network layer
from tensorflow.keras.layers import Dense,Flatten
#Faltten-used fot flattening the input or change the dimension
from tensorflow.keras.layers import Conv2D,MaxPooling2D,Dropout #Convolutional layer
#MaxPooling2D-for downsampling the image
from keras.preprocessing.image import ImageDataGenerator

In [2]:
!pip install keras==2.2.4

     |████████████████████████████�██▌| 307 kB 9.0 MB/s eta 0:00:01�███| 312 kB 9.0 MB/s 
     |████████████████████████████████| 50 kB 8.9 MB/s  eta 0:00:01
  Attempting uninstall: keras
    Found existing installation: keras 2.7.0
    Uninstalling keras-2.7.0:
      Successfully uninstalled keras-2.7.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.7.2 requires keras<2.8,>=2.7.0, but you have keras 2.2.4 which is incompatible.


In [3]:
!pip install tensorflow

     |████████████████████████████████| 1.3 MB 8.7 MB/s eta 0:00:01
  Attempting uninstall: keras
    Found existing installation: Keras 2.2.4
    Uninstalling Keras-2.2.4:
      Successfully uninstalled Keras-2.2.4


In [4]:
import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='W0UCSR6XpLZeifxNGh3Oe1RlOt3i_sz_w6yDpD61rzpc',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.eu.cloud-object-storage.appdomain.cloud')

bucket = 'training-donotdelete-pr-lxglhnda12gseh'
object_key = 'Dataset.zip'

streaming_body_1 = cos_client.get_object(Bucket=bucket, Key=object_key)['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/

In [5]:
from io import BytesIO
import zipfile
unzip=zipfile.ZipFile(BytesIO(streaming_body_1.read()),'r')
file_paths=unzip.namelist()
for path in file_paths:
    unzip.extract(path)

In [6]:
#setting parameter for Image Data agumentation to the training data
train_datagen = ImageDataGenerator(rescale=1./255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True)

In [7]:
#Image Data agumentation to the testing data
test_datagen=ImageDataGenerator(rescale=1./255)

In [8]:
#performing data agumentation to train data
x_train = train_datagen.flow_from_directory(
    r'Dataset/TRAIN_SET',
    target_size=(64, 64),batch_size=113,color_mode='rgb',class_mode='sparse')

Found 2626 images belonging to 5 classes.


In [9]:
#performing data agumentation to test data
x_test = test_datagen.flow_from_directory(
    r'Dataset/TEST_SET',
    target_size=(64, 64),batch_size=113,color_mode='rgb',class_mode='sparse')

Found 1055 images belonging to 5 classes.


In [10]:
print(x_train.class_indices)#checking the number of classes

{'APPLES': 0, 'BANANA': 1, 'ORANGE': 2, 'PINEAPPLE': 3, 'WATERMELON': 4}


In [11]:
print(x_test.class_indices)#checking the number of classes

{'APPLES': 0, 'BANANA': 1, 'ORANGE': 2, 'PINEAPPLE': 3, 'WATERMELON': 4}


# Model Building

In [12]:
from collections import Counter as c
c(x_train .labels)

Counter({0: 606, 1: 445, 2: 479, 3: 621, 4: 475})

In [13]:
# Initializing the CNN
classifier = Sequential()

In [14]:
# First convolution layer and pooling
classifier.add(Conv2D(32, (3, 3), input_shape=(64, 64, 3), activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2, 2)))

In [15]:
# Second convolution layer and pooling
classifier.add(Conv2D(32, (3, 3), activation='relu'))

In [16]:
# input_shape is going to be the pooled feature maps from the previous convolution layer
classifier.add(MaxPooling2D(pool_size=(2, 2)))

In [17]:
# Flattening the layers
classifier.add(Flatten())

In [18]:
# Adding a fully connected layer
classifier.add(Dense(units=128, activation='relu'))
classifier.add(Dense(units=113, activation='softmax')) # softmax for more than 2
classifier.summary()#summary of our model

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 62, 62, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 31, 31, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 29, 29, 32)        9248      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 14, 14, 32)       0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 6272)              0         
                                                                 
 dense (Dense)               (None, 128)               8

In [19]:
# Compiling the CNN
# categorical_crossentropy for more than 2
classifier.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
classifier.fit_generator(
        generator=x_train,steps_per_epoch = len(x_train),
        epochs=100, validation_data=x_test,validation_steps = len(x_test))# No of images in test set

/tmp/wsuser/ipykernel_208/2719268462.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  classifier.fit_generator(


Epoch 1/100
24/24 [==============================] - 23s 893ms/step - loss: 1.1090 - accuracy: 0.6691 - val_loss: 0.1841 - val_accuracy: 0.9573
Epoch 2/100
24/24 [==============================] - 20s 820ms/step - loss: 0.1428 - accuracy: 0.9615 - val_loss: 0.1242 - val_accuracy: 0.9725
Epoch 3/100
24/24 [==============================] - 22s 911ms/step - loss: 0.0426 - accuracy: 0.9905 - val_loss: 0.1691 - val_accuracy: 0.9232
Epoch 4/100
24/24 [==============================] - 22s 919ms/step - loss: 0.0111 - accuracy: 0.9977 - val_loss: 0.1727 - val_accuracy: 0.9109
Epoch 5/100
24/24 [==============================] - 21s 841ms/step - loss: 0.0099 - accuracy: 0.9977 - val_loss: 0.1142 - val_accuracy: 0.9384
Epoch 6/100
24/24 [==============================] - 20s 824ms/step - loss: 0.0027 - accuracy: 1.0000 - val_loss: 0.1191 - val_accuracy: 0.9261
Epoch 7/100
24/24 [==============================] - 21s 852ms/step - loss: 0.0016 - accuracy: 1.0000 - val_loss: 0.2379 - val_accuracy:

24/24 [==============================] - 21s 856ms/step - loss: 2.5389e-05 - accuracy: 1.0000 - val_loss: 0.0742 - val_accuracy: 0.9706
Epoch 57/100
24/24 [==============================] - 22s 901ms/step - loss: 1.5398e-05 - accuracy: 1.0000 - val_loss: 0.0762 - val_accuracy: 0.9706
Epoch 58/100
24/24 [==============================] - 20s 839ms/step - loss: 2.1692e-05 - accuracy: 1.0000 - val_loss: 0.0825 - val_accuracy: 0.9687
Epoch 59/100
24/24 [==============================] - 22s 893ms/step - loss: 2.8358e-05 - accuracy: 1.0000 - val_loss: 0.0920 - val_accuracy: 0.9621
Epoch 60/100
24/24 [==============================] - 22s 888ms/step - loss: 2.0740e-05 - accuracy: 1.0000 - val_loss: 0.0848 - val_accuracy: 0.9678
Epoch 61/100
24/24 [==============================] - 21s 861ms/step - loss: 1.9491e-05 - accuracy: 1.0000 - val_loss: 0.0821 - val_accuracy: 0.9697
Epoch 62/100
24/24 [==============================] - 21s 872ms/step - loss: 2.6491e-05 - accuracy: 1.0000 - val_loss: 

In [ ]:
# Save the model
classifier.save('nutrition.h5')

In [ ]:
classifier_json = classifier.to_json()
with open("classifier-bw.json", "w") as json_file:
    json_file.write(classifier_json)

In [ ]:
!tar -zcvf Nutrition.tgz nutrition.h5

In [ ]:
!pip install ibm_watson_machine_learning

In [ ]:
from ibm_watson_machine_learning import APIClient

API_KEY = "K7_NvMBBBBT1ntSd5ZB54lWu1CwfQZExZit4tbh1U0Xg"

wml_credentials = {
    "url": "https://eu-de.ml.cloud.ibm.com",
    "apikey": API_KEY
}

client = APIClient(wml_credentials)

In [ ]:
def guid_space_name(client,space_name):
    space=client.spaces.get_details()
    return(next(item for item in space['resources'] if item['entity']['name']==space_name)['metadata']['id'])

In [ ]:
space_uid="37529bdc-b05b-46e4-ad0f-2f9f2a66bf4a"
print("Space UID "+space_uid)

In [ ]:
client.set.default_space(space_uid)

In [ ]:
client.software_specifications.list()

In [ ]:
software_spec_uid = client.software_specifications.get_uid_by_name("runtime-22.1-py3.9")
software_spec_uid

In [ ]:
model_details = client.repository.store_model(model="Nutrition.tgz", meta_props={
    client.repository.ModelMetaNames.NAME: "Models",
    client.repository.ModelMetaNames.TYPE: "tensorflow_2.7",
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: software_spec_uid
})

model_id = client.repository.get_model_id(model_details)

In [ ]:
model_id

In [ ]:
client.repository.download(model_id, "model.tar.gz")

In [ ]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np

In [ ]:
img=image.load_img(r"Dataset/photo-1589820296156-2454bb8a6ad1.jpg",target_size=(64,64))

In [ ]:
img

In [ ]:
x=image.img_to_array(img)#conversion image into array

In [ ]:
x

In [ ]:
x.ndim

In [ ]:
x=np.expand_dims(x,axis=0) #expand the dimension

In [ ]:
x.ndim

In [ ]:
pred = classifier.predict(x)

In [ ]:
pred

In [ ]:
labels=['APPLES','BANANA','ORANGE','PINEAPPLE','WATERMELON']

In [ ]:
labels[np.argmax(pred)]